In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
if not "../scripts" in sys.path:
    sys.path.append("../scripts")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
metadata = pd.read_csv("../data/metadata.csv")
puzzle_dir = "../data/puzzles"

In [5]:
from utils import *
puzzles = [
    Puzzle(int(x), os.path.join(puzzle_dir, x)) for x in os.listdir(puzzle_dir) if int(x) in metadata.puzzle_id]
puzzles = sorted(puzzles, key=lambda x: x.id)

In [6]:
#puzzle = puzzles[1]
#puzzle_data = read_puzzle(puzzle)

In [7]:
puzzle_size = {i: (h, w) for i, w, h in metadata.values}
#size = puzzle_size.get(puzzle.id)

In [8]:
LABELS = ["down", "up", "right", "left"]

label_ids = {x: i for i,x in enumerate(LABELS)}

In [9]:
# W = df['up'].sum()
# H = df['left'].sum()

In [10]:
# upleft = df.loc[df.up & df.left, 'path'].values[0]
# upright = df.loc[df.up & df.right, 'path'].values[0]
# downleft = df.loc[df.down & df.left, 'path'].values[0]
# downright = df.loc[df.down & df.right, 'path'].values[0]

# left = df.loc[df.left & ~df.up & ~df.down, 'path'].values
# right = df.loc[df.right & ~df.up & ~df.down, 'path'].values

# up = df.loc[df.up & ~df.left & ~df.right, 'path'].values
# down = df.loc[df.down & ~df.left & ~df.right, 'path'].values

# inner_nodes = df.loc[~(df.up&df.down&df.left&df.right), 'path'].values

In [11]:
from itertools import combinations

def compute_scores(df, label_ids, puzzle_data={}):
    all_scores_down = dict()
    all_scores_right = dict()
    all_paths = list(df.loc[:, "path"].values) #+ [downleft, upleft]
    align_type = 'best'

    label_a = 'down'#'right'
    label_b = 'up'#'left'
    for i, j in combinations(all_paths, 2):
        p = puzzle_data[i]
        q = puzzle_data[j]
        a = p.outer[label_ids[label_a]]
        b = q.inner[label_ids[label_b]]
        if a is not None and b is not None:
            # compare
            #plt.imshow(a)
            #plt.show()
            #plt.imshow(b)
            #plt.show()
            a = outer_transform[label_a](a)
            b = inner_transform[label_b](b)
            pair = (i, j)
            all_scores_down[pair] = match(a, b, align_type=align_type)
            #break
        a = p.inner[label_ids[label_a]]
        b = q.outer[label_ids[label_b]]
        if a is not None and b is not None:
            a = inner_transform[label_a](a)
            b = outer_transform[label_b](b)
            pair = (i, j)
            all_scores_down[pair] = match(a, b, align_type=align_type)
            #break
        # reversed
        a = q.inner[label_ids[label_a]]
        b = p.outer[label_ids[label_b]]
        if a is not None and b is not None:
            # compare
            a = inner_transform[label_a](a)
            b = outer_transform[label_b](b)
            pair = (j, i)
            all_scores_down[pair] = match(a, b, align_type=align_type)
            #break
        a = q.outer[label_ids[label_a]]
        b = p.inner[label_ids[label_b]]
        if a is not None and b is not None:
            a = outer_transform[label_a](a)
            b = inner_transform[label_b](b)
            pair = (j, i)
            all_scores_down[pair] = match(a, b, align_type=align_type)

    label_a = 'right'
    label_b = 'left'
    for i, j in combinations(all_paths, 2):
        p = puzzle_data[i]
        q = puzzle_data[j]
        a = p.outer[label_ids[label_a]]
        b = q.inner[label_ids[label_b]]

        if a is not None and b is not None:
            # compare
            #plt.imshow(a)
            #plt.show()
            #plt.imshow(b)
            #plt.show()
            a = outer_transform[label_a](a)
            b = inner_transform[label_b](b)
            pair = (i, j)
            all_scores_right[pair] = match(a, b, align_type=align_type)
            #break
        a = p.inner[label_ids[label_a]]
        b = q.outer[label_ids[label_b]]
        if a is not None and b is not None:
            a = inner_transform[label_a](a)
            b = outer_transform[label_b](b)
            pair = (i, j)
            all_scores_right[pair] = match(a, b, align_type=align_type)
            #break
        # reversed
        a = q.inner[label_ids[label_a]]
        b = p.outer[label_ids[label_b]]
        if a is not None and b is not None:
            # compare
            a = inner_transform[label_a](a)
            b = outer_transform[label_b](b)
            pair = (j, i)
            all_scores_right[pair] = match(a, b, align_type=align_type)
            #break
        a = q.outer[label_ids[label_a]]
        b = p.inner[label_ids[label_b]]
        if a is not None and b is not None:
            a = outer_transform[label_a](a)
            b = inner_transform[label_b](b)
            pair = (j, i)
            all_scores_right[pair] = match(a, b, align_type=align_type)
    return all_scores_down, all_scores_right

def compute_edges(df, label_ids, puzzle_data={}):
    all_scores_down, all_scores_right = compute_scores(df, label_ids, puzzle_data=puzzle_data)

    scores = dict()
    for (s, e), w in all_scores_down.items():
        if not s in scores:
            scores[s] = {'down': [], 'right': []}
        scores[s]['down'].append((e, w))

    for (s, e), w in all_scores_right.items():
        if not s in scores:
            scores[s] = {'down': [], 'right': []}
        scores[s]['right'].append((e, w))
    return all_scores_down, all_scores_right, scores



In [12]:
def add_node2(paths, scores, verbose=False, limit_paths=[], max_weight=200):
    for path, w0, w2 in paths:
        if verbose:
            print(path)
        s = path[-1]
        if not s in scores:
            continue
        for e, w in scores[s]['right']:
            if e in path:
                continue
            if len(limit_paths) > 0:
                if e not in limit_paths:
                    continue
            if w > max_weight:
                continue
            yield path + [e], w0+w, w2 + [w]

def get_lines(df, scores={}, max_paths=100):
    W = df['up'].sum()
    H = df['left'].sum()
    
    upleft = df.loc[df.up & df.left, 'path'].values[0]
    upright = df.loc[df.up & df.right, 'path'].values[0]
    downleft = df.loc[df.down & df.left, 'path'].values[0]
    downright = df.loc[df.down & df.right, 'path'].values[0]
    
    left = df.loc[df.left & ~df.up & ~df.down, 'path'].values
    right = df.loc[df.right & ~df.up & ~df.down, 'path'].values
    up = df.loc[df.up & ~df.left & ~df.right, 'path'].values
    down = df.loc[df.down & ~df.left & ~df.right, 'path'].values
    
    inner_nodes = df.loc[~(df.up&df.down&df.left&df.right), 'path'].values
    
    MAX_PATHS = max_paths
    lines = []
    # add first line
    line_info = [([upleft], 0, [])]
    for i in range(1, W-1):
        line_info = list(add_node2(line_info, scores, limit_paths=up))
        line_info = sorted(line_info, key=lambda x: x[1])
        if MAX_PATHS > 0:
            line_info = line_info[:MAX_PATHS]
    line_info = list(add_node2(line_info, scores, limit_paths=[upright]))
    line_info = sorted(line_info, key=lambda x: x[1])#[:100]
    if MAX_PATHS > 0:
        line_info = line_info[:MAX_PATHS]
    lines.append(line_info)

    for l in left:
        line_info = [([l], 0, [])]
        for i in range(1, W-1):
            line_info = list(add_node2(line_info, scores, limit_paths=inner_nodes))
            line_info = sorted(line_info, key=lambda x: x[1])
            if MAX_PATHS > 0:
                line_info = line_info[:MAX_PATHS]
        line_info = list(add_node2(line_info, scores, limit_paths=right))
        line_info = sorted(line_info, key=lambda x: x[1])
        if MAX_PATHS > 0:
            line_info = line_info[:MAX_PATHS]
        lines.append(line_info)

    line_info = [([downleft], 0, [])]
    for i in range(1, W-1):
        line_info = list(add_node2(line_info, scores, limit_paths=down))
        line_info = sorted(line_info, key=lambda x: x[1])
        if MAX_PATHS > 0:
            line_info = line_info[:MAX_PATHS]
    line_info = list(add_node2(line_info, scores, limit_paths=[downright]))
    line_info = sorted(line_info, key=lambda x: x[1])
    if MAX_PATHS > 0:
        line_info = line_info[:MAX_PATHS]
    lines.append(line_info)
    return lines

In [13]:
from tqdm.notebook import tqdm


def add_line(sel_lines, lines, all_scores_down={}, text=""):
    with tqdm(total=len(sel_lines), leave=False) as pbar:
        i = 0
        for prev_lines, prev_w, wdetailed in sel_lines:
            seen_paths = [x for xs in prev_lines for x in xs[0]]
            seen_paths = set(seen_paths)
            if len(prev_lines[-1])!=3:
                print(prev_lines[-1])
            prev_line, prev_weight, cum_weight = prev_lines[-1]
            for data in lines:
                line, w_line, w_c = data
                if np.any([ (path in seen_paths) for path in line]):
                    continue
                score_w = 0
                for a, b in zip(prev_line, line):
                    if not (a, b) in all_scores_down:
                        score_w = -1
                        break
                    score_w += all_scores_down[(a, b)]
                if score_w < 0:
                    continue
                yield prev_lines+[data], score_w + prev_w + w_line, wdetailed+[score_w, w_line]
            i += 1
            if i >= 20:
                pbar.update(i)
                pbar.set_description(text)
                i = 0
        if i > 0:
            pbar.update(i)
            
                    

In [14]:
def combine_lines(df, lines, all_scores_down={}, text=""):
    W = df['up'].sum()
    H = df['left'].sum()
    starts = [([l], l[1], [l[1]]) for l in lines[0]]
    middle = [x for xs in lines[1:-1] for x in xs]
    for j in range(1, H-1):
        starts = list(add_line(starts, middle, all_scores_down=all_scores_down,
                               text=f"{text} {j}"))
        starts = sorted(starts, key=lambda x: x[1])
        starts = starts[:1000]
    starts = list(add_line(starts, lines[-1], all_scores_down=all_scores_down,
                           text=f"{text} {H-1}"))
    starts = sorted(starts, key=lambda x: x[1])
    return starts

def solve_puzzle(df, all_scores_down, scores={}, max_paths=100, text=""):
    lines = get_lines(df, scores=scores, max_paths=max_paths)
    starts = combine_lines(df, lines, all_scores_down=all_scores_down, text=text)
    return starts

In [15]:
def draw_with_offset(target, piece, offset):
    h, w = offset
    pos = np.argwhere(piece.contour>0)
    hcoord = pos[:, 0]
    wcoord = pos[:, 1]
    target[hcoord+h, wcoord+w, :] = piece.img[hcoord, wcoord, :]

def draw_puzzle(collection, size, H, W, puzzle_data={}):
    target_img = np.zeros(size+(4,), dtype=np.uint8)
    h, w = size
    hstep = h/collection.height
    wstep = w/collection.width
    offsets_table = []
    for i, line in enumerate(collection.field):
        offsets = []
        for j, path in enumerate(line):
            offset_h = hstep/2 + hstep*i
            offset_w = wstep/2 + wstep*j
            h_, w_ = puzzle_data[path].img.shape[:2]
            offset_h -= h_/2
            offset_w -= w_/2
            offset_h = max(int(offset_h), 0)
            offset_w = max(int(offset_w), 0)
            if i == H-1:
                offset_h = h - h_
            if j == W - 1:
                offset_w = w - w_
            offsets.append((offset_h, offset_w))
            draw_with_offset(target_img, puzzle_data[path], (offset_h, offset_w))
        offsets_table.append(offsets)
    return target_img

In [16]:
def draw_puzzle_to_file(df, puzzle_data, solutions, size=(0, 0), savedir=".", puzzle_id=0):
    W = df['up'].sum()
    H = df['left'].sum()
    collection = Collection(H, W, puzzle_data)
    for i, line in enumerate(solutions[0][0]):
        for j, x in enumerate(line[0]):
            collection.fill(i, j, x)

    cv2.imwrite(os.path.join(savedir, f"{puzzle_id}.jpg"),
                draw_puzzle(collection, size, H, W, puzzle_data=puzzle_data)[:, :, :3])

In [17]:
def find_solution_for(puzzle, puzzle_size, savedir="temp", max_paths=100, force=False):
    path = os.path.join(savedir, f"{puzzle.id}.jpg")
    if os.path.exists(path) and not force:
        return
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    size = puzzle_size.get(puzzle.id)
    puzzle_data = read_puzzle(puzzle)
    item_info = []
    for path, item in puzzle_data.items():
        info = {l: x is None and y is None for x, y, l in zip(item.inner, item.outer, LABELS)}
        info['path'] = path
        item_info.append(info)
    df = pd.DataFrame(item_info)
    all_scores_down, all_scores_right, scores = compute_edges(df, label_ids, puzzle_data=puzzle_data)
    text = f"Solving {puzzle.id}:"
    starts = solve_puzzle(df, all_scores_down, scores=scores, max_paths=max_paths, text=text)
    if len(starts) == 0:
        starts = solve_puzzle(df, all_scores_down, scores=scores, max_paths=1000, text=text)
    
    draw_puzzle_to_file(df, puzzle_data, starts, size=size, savedir=savedir, puzzle_id = puzzle.id)
    

In [ ]:
for puzzle in tqdm(puzzles):
    find_solution_for(puzzle, puzzle_size)